In [21]:
#przyklad weak supervised algorithm

In [22]:
import tensorflow as tf

In [23]:
import numpy as np

In [24]:
import matplotlib.pyplot as plt

In [25]:
%matplotlib inline

In [26]:
from sklearn.utils import shuffle

In [27]:
import Io_tf_binary_general as io

In [28]:
#utworze teraz dane jakies
#tutaj zachecam do machania tymi domyslnymi wartosciami. Tu niestety tego nie narysowalem, ale 
# to tworzy takie wlasnie kolko w kolku na plaszczyxnie.
def kolko_w_kolku(klasa,sig1=0.4,sig2=0.25,R2=0.8,N=1000):
    """to ma utworzyc dane ktore posluza mi do sprawdzenia czy i jak dziala model
    klasa to 0 lub 1"""
    
    if klasa==1:
        x=np.random.normal(0,sig1)
        y=np.random.normal(0,sig1)
        #features.append([x,y])
        return {"pierwsza":x,"inna_wlasnosc":y}
    else:
        alpha=np.random.uniform()*2*np.pi
        delr=np.random.normal(0,sig2)
        r=R2+delr
        x=np.cos(alpha)*r
        y=np.sin(alpha)*r
        #features.append([x,y])
        return {"pierwsza":x,"inna_wlasnosc":y}

In [29]:
p0_train=0.3
p1_train=0.7
p0_validate=0.2
p1_validate=0.8
#to pierwse to prawdopodobienstwo, ze cos z pierwszej populacji jest naprawdę 1
# to drugie to prawdopodobiensto, ze cos z drugiej populacji jest naprawde 1

In [30]:
"""Tworze datasety ktore odpowiadaja koejno danym treningowym skladajacym sie
 z labelek takich, ze jak jakas wskazuje 0 to z prawdopodobienstwem 0.3 jednak jest jedynka (rozpadem
 higgsa), zas jak wskazuje 1 to z prawdopodobienstwem 0.7 jest rzeczywiscie rozpadem. To teraz pisalem o 
 tym datascie "weak_train". potem tworze jeszcze weak_validate ktory jest mniejszy ale idea jest taka,
 ze na weak_validate znamy skad inond te prawdopodobienstwa, a na weak_train nie znamy ich.
 weak_true tworze tylko po to, by moc sprawdzic, czy policzona auc na danych weak_validate zgodnie
 z ta praca ktora na poczatku nam przesylal Kalinowski o weak supervised learning zgadza sie
 z rzeczywistoscia"""

'Tworze datasety ktore odpowiadaja koejno danym treningowym skladajacym sie\n z labelek takich, ze jak jakas wskazuje 0 to z prawdopodobienstwem 0.3 jednak jest jedynka (rozpadem\n higgsa), zas jak wskazuje 1 to z prawdopodobienstwem 0.7 jest rzeczywiscie rozpadem. To teraz pisalem o \n tym datascie "weak_train". potem tworze jeszcze weak_validate ktory jest mniejszy ale idea jest taka,\n ze na weak_validate znamy skad inond te prawdopodobienstwa, a na weak_train nie znamy ich.\n weak_true tworze tylko po to, by moc sprawdzic, czy policzona auc na danych weak_validate zgodnie\n z ta praca ktora na poczatku nam przesylal Kalinowski o weak supervised learning zgadza sie\n z rzeczywistoscia'

In [31]:
pisacz=io.Io_tf_binary_general("weak_train",'w')
for i in range(10000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=int(np.random.uniform()<p0_train)
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=int(np.random.uniform()<p1_train)
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [32]:
pisacz=io.Io_tf_binary_general("weak_validate",'w')
for i in range(1000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=int(np.random.uniform()<p0_validate)
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=int(np.random.uniform()<p1_validate)
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [33]:
pisacz=io.Io_tf_binary_general("weak_true",'w')
for i in range(1000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=klasa
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=klasa
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [34]:
czytacz=io.Io_tf_binary_general("weak_validate",'r')
dataset=czytacz.read()
BATCH_SIZE=2
zbachowany=dataset.shuffle(1000).repeat().batch(BATCH_SIZE)
iterator = zbachowany.make_one_shot_iterator()
para=iterator.get_next()

with tf.Session() as sess:
    for i in range(3):
        print(sess.run(para))

{'pierwsza': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'inna_wlasnosc': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}
({'inna_wlasnosc': array([[0.22005723],
       [0.22612055]], dtype=float32), 'pierwsza': array([[-0.6884664 ],
       [-0.60397434]], dtype=float32)}, array([0, 0]))
({'inna_wlasnosc': array([[-0.7017992 ],
       [-0.29339036]], dtype=float32), 'pierwsza': array([[-0.02933752],
       [ 0.5992255 ]], dtype=float32)}, array([0, 1]))
({'inna_wlasnosc': array([[-0.09941217],
       [-0.10785457]], dtype=float32), 'pierwsza': array([[-0.35607988],
       [-0.7815257 ]], dtype=float32)}, array([0, 0]))


In [35]:

import Dnn_uniwersalny as dnn

In [36]:
model=dnn.Dnn_uniwersalny("weak_train",[10],"weak_estimator")

{'pierwsza': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'inna_wlasnosc': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'weak_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f38bbe75eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
model.make_model()

AttributeError: 'Dnn_uniwersalny' object has no attribute 'make_model'

In [ ]:
model.train() #trenujemy na "weak_train"

In [ ]:
auc,xxxx,yyyy,zzz=model.evaluate_jak_z_pracy(0.2,0.8,folder="weak_validate")
auc #auc policzona na weak_validate zgodnie z praca 

In [ ]:
model.evaluate(folder="weak_true")
    #auc policzona zgodnie z tym, jak to sie liczy normalnie gdy ma sie dobrze oznaczone dane gdzie
    # na pewno byl higgs lub na pewno nie bylo

In [ ]:
"""Tutaj nalezy zwrocic swoja uwage na to, ze auc policzone na danych co do ktorych wiemy dokladnie czy
sa rozpadem czy nie jest podobne do auc policzonego na podstawie labelek co do ktorych znamy tylko prawdopodobienstwo
czy sa rozpadem"""
    